In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import jieba
import jieba.posseg as psg
from gensim import corpora, models
from jieba import analyse
import functools


In [2]:
all_docs_df = pd.read_csv('../../data/chusai/all_docs.txt', sep='\001', header=None)
all_docs_df.columns = ['id', 'title', 'text']
all_docs_df['title'] = all_docs_df['title'].astype(str)
all_docs_df['text'] = all_docs_df['text'].astype(str)
print(all_docs_df.info())
print(all_docs_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108295 entries, 0 to 108294
Data columns (total 3 columns):
id       108295 non-null object
title    108295 non-null object
text     108295 non-null object
dtypes: object(3)
memory usage: 2.5+ MB
None
        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...  
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...  
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...  
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...  
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...  


In [4]:
all_docs_df['title_text'] = all_docs_df['title'] + '。' + all_docs_df['text']
print(all_docs_df.head())


        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                      title_list  \
0  [林志颖, 老婆, 深夜, 面膜, 睫毛, 太长, 吓人]   
1          [夸杨, 身材, 杨幂, 回复, 太精彩]   
2            [年轻, 风华绝代, 无人, 送祝福]   
3     [林心如, 曝霍, 建华, 私生活, 老公, 人气]   
4     [TVB, 颜值, 担当, 照曝光, 发现, 真老]   

                                          title_text  
0  林志颖老婆深夜敷面膜，睫毛太长好吓人。早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩...  
1  小s夸杨幂身材好，杨幂回复太精彩了！。翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，...  
2  年轻时的她风华绝代，现却无人送祝福。上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之...  
3  林心如屡曝霍建华私生活被

In [5]:
#定义textrank函数
def textrank_extract(text, pos=False, keyword_num=10):
    textrank = analyse.textrank
    keywords = textrank(text, keyword_num)
    return keywords

all_docs_df['keywords'] = all_docs_df['title_text'].map(lambda x: textrank_extract(x, False, 5))
print(all_docs_df.head())


        id                title  \
0  D000001   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003    年轻时的她风华绝代，现却无人送祝福   
3  D000004   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                      title_list  \
0  [林志颖, 老婆, 深夜, 面膜, 睫毛, 太长, 吓人]   
1          [夸杨, 身材, 杨幂, 回复, 太精彩]   
2            [年轻, 风华绝代, 无人, 送祝福]   
3     [林心如, 曝霍, 建华, 私生活, 老公, 人气]   
4     [TVB, 颜值, 担当, 照曝光, 发现, 真老]   

                                          title_text              keywords  
0  林志颖老婆深夜敷面膜，睫毛太长好吓人。早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩...  [面膜, 睫毛, 安利, 了解, 大家]  
1  小s夸杨幂身材好，杨幂回复太精彩了！。翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，...  [直到, 气度, 古典, 带有, 典型]  
2  年

In [9]:
sample_df = pd.read_csv('../../result/chusai/sample.csv', encoding='ISO-8859-1')
print(len(sample_df))
sample_df = pd.merge(sample_df, all_docs_df, on='id', how='left')
print(sample_df.head())


107295
        id label1 label2                title  \
0  D000001     ÎÞ     ÎÞ   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002     ÎÞ     ÎÞ   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003     ÎÞ     ÎÞ    年轻时的她风华绝代，现却无人送祝福   
3  D000004     ÎÞ     ÎÞ   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005     ÎÞ     ÎÞ  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                      title_list  \
0  [林志颖, 老婆, 深夜, 面膜, 睫毛, 太长, 吓人]   
1          [夸杨, 身材, 杨幂, 回复, 太精彩]   
2            [年轻, 风华绝代, 无人, 送祝福]   
3     [林心如, 曝霍, 建华, 私生活, 老公, 人气]   
4     [TVB, 颜值, 担当, 照曝光, 发现, 真老]   

                                          title_text              keywords  
0  林志颖老婆深夜敷面膜，睫毛太长好吓人。早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩...  [面膜, 睫毛, 安利, 

In [10]:
def get_top_n_word(result_list, n):
    if len(result_list) < n:
        return '无'
    else:
        return result_list[n - 1]
    
sample_df['label1'] = sample_df['keywords'].map(lambda x: get_top_n_word(x, 1))
sample_df['label2'] = sample_df['keywords'].map(lambda x: get_top_n_word(x, 2))
print(sample_df.head())


        id label1 label2                title  \
0  D000001     面膜     睫毛   林志颖老婆深夜敷面膜，睫毛太长好吓人   
1  D000002     直到     气度   小s夸杨幂身材好，杨幂回复太精彩了！   
2  D000003     选择     希望    年轻时的她风华绝代，现却无人送祝福   
3  D000004     粉丝     女儿   林心如屡曝霍建华私生活被怼蹭老公人气   
4  D000005     观众     觉得  曾是TVB颜值担当，近照曝光发现真老了   

                                                text  \
0  早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩掩的林志颖老婆低调探班，总让人觉得格外...   
1  翩若惊鸿，婉若游龙。曹植形容洛神的这两句，实在太抽象，以至于始终寻不到承受对象。直到在《大军...   
2  上个世纪香港影视界涌现出了不少高颜值女星，在《大话西游之月光宝盒》中饰演春三十娘和蜘蛛精的蓝...   
3  霍建华林心如1905电影网讯近日，林心如在接受采访时爆料称老公霍建华会主动向女儿索吻，笑称他...   
4  不知道有多少人是看TVB剧集长大的，小时候我每一天晚上都会守着电视看TVB剧集的。可以说对于...   

                      title_list  \
0  [林志颖, 老婆, 深夜, 面膜, 睫毛, 太长, 吓人]   
1          [夸杨, 身材, 杨幂, 回复, 太精彩]   
2            [年轻, 风华绝代, 无人, 送祝福]   
3     [林心如, 曝霍, 建华, 私生活, 老公, 人气]   
4     [TVB, 颜值, 担当, 照曝光, 发现, 真老]   

                                          title_text              keywords  
0  林志颖老婆深夜敷面膜，睫毛太长好吓人。早年林志颖带kimi上《爸爸去哪儿》的时候，当时遮遮掩...  [面膜, 睫毛, 安利, 了解, 大家]

In [11]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../../result/chusai/%s.csv' % fileName, header=True, index=False)
    
exportResult(sample_df[['id', 'label1', 'label2']], 'textrank_baseline_9_3_3')
